In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
import warnings
import itertools

from pyramid.arima import auto_arima
import plotly.plotly as ply
warnings.filterwarnings("ignore")

df = pd.read_csv("House1_1min_Filled.csv",index_col = 0)
df.head()
df.index = pd.to_datetime(df.index)
df = df.resample("H", label='right', closed='right').sum()
df.head()

In [ ]:
df = df.iloc[:,:7]
df.head()

In [ ]:
household = pd.DataFrame(df.sum(axis=1),columns = ["Total Electricity"] )
household

In [ ]:
X = household["Total Electricity"].values
size = int(len(X) * 0.75)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
stepwise_model = auto_arima(history, start_p=0, start_q=0, max_p=5,max_q=5, start_P=0, start_Q=0, max_P=5,max_Q=5, m = 24, seasonal=True,
                           trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)


In [ ]:


from pandas import datetime
from matplotlib import pyplot
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX
 

X = household.values
size = int(len(X) * 0.75)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()


# Sarima: Rolling Sarima
for t in range(len(test)):
    model=SARIMAX(history,order=(3,0,1),seasonal_order=(1,0,1,24),enforce_invertibility=False, enforce_stationarity = False)
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))
error = mean_absolute_percentage_error(test, predictions)
print('Test MAPE: %.3f' % error)

In [ ]:
import numpy as np

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = list(y_true), list(y_pred)
    l = []
    for i in range(len(y_true)):
        a = abs(y_true[i] - y_pred[i])
        b = a/y_pred[i]
        l.append(b)
    print(sum(l))
    print(len(l))
    return (sum(l)/len(l))*100

        

In [ ]:
mean_absolute_percentage_error(test, predictions)

In [ ]:
from plotly import __version__
%matplotlib inline
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot 
init_notebook_mode(connected=True)

init_notebook_mode(connected=True)
cf.go_offline()
future_forecast = pd.DataFrame(predictions,index = household["Total Electricity"][int(len(household.values) * 0.85):].index,columns=['Prediction'])
pd.concat([household["Total Electricity"],future_forecast],axis=1).iplot()